In [17]:
import glob

exclude = ['./CLAUDE.md','./progress-tracker.md','./project-description.md']
list_of_files = glob.glob('meridian-islands/**/*.md') 
filtered_list = []
for a in list_of_files:
    if a not in exclude:
        filtered_list.append(a)

filtered_list = sorted(filtered_list)

current_list = filtered_list[0]
print(current_list)

markdown_string = " "

with open(current_list, "r", encoding="utf-8") as f:
    converted_text = f.read() 

with open("first.txt", "w", encoding="utf-8") as text_file:
    text_file.write(converted_text)

meridian-islands/01-foundation/world-bible.md


In [ ]:
from dataclasses import dataclass

@dataclass
class Chunk:
        text: str
        chunk_id: int
        start_pos: int
        end_pos: int
        char_count: int
        origin_file: str


class Chunker:
    def __init__(self,chunk_size):
        self.chunk_size = chunk_size
       
    def chunk(self, file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            converted_text = f.read()

        chunks = []
        i=0
        chunk_size = self.chunk_size
        for start in (range(0, len(converted_text), chunk_size)):
            end = min(start+chunk_size, len(converted_text))
            chunk = converted_text[start:end]
            chunks.append(
                Chunk(
                    text=chunk,
                    chunk_id=i+1,
                    start_pos=start,
                    end_pos=end-1,
                    char_count=len(chunk),
                    origin_file=current_list
                )
            )
            i += 1
        return chunks


In [51]:
chunker = Chunker(200)
chunks = chunker.chunk("meridian-islands/01-foundation/world-bible.md")

for c in chunks[:2]:
    print(c)

Chunk(text='# Meridian Islands World Bible\n*Master Reference Document - All content must align with these established facts*\n\n## Archipelago Overview\n\nThe Meridian Islands are a sovereign island nation located in', chunk_id=1, start_pos=0, end_pos=199, char_count=200, origin_file='meridian-islands/01-foundation/world-bible.md')
Chunk(text=' the South Pacific Ocean, approximately 2,100 kilometers east of Australia and 1,800 kilometers north of New Zealand. The archipelago consists of six islands: four main inhabited islands and two small', chunk_id=2, start_pos=200, end_pos=399, char_count=200, origin_file='meridian-islands/01-foundation/world-bible.md')
